# Napari raw image data and tracks viewer
Works as of Dec 2020 with Napari version no. 0.4.0 **with btrack plug-in installed**

Author: `nathan.day.16@ucl.ac.uk`

### Load necessary modules and functions

In [2]:
import napari
import btrack
import numpy as np
from skimage.io import imread
from btrack.utils import tracks_to_napari
print("Napari version no.:", napari.__version__)

def find_apoptosis_time(target_track, index): ### if index is set to True then the index of the apoptotic time (wrt target_track) is returned
    for i, j in enumerate(target_track.label):
        if j == 'APOPTOSIS' and target_track.label[i+1] == 'APOPTOSIS' and target_track.label[i+2] == 'APOPTOSIS': # and target_track.label[i+3] =='APOPTOSIS' and target_track.label[i+4] =='APOPTOSIS':
            apop_index = i
            break
    apop_time = target_track.t[apop_index]
    if index == True: 
        return apop_index
    else: 
        return apop_time

Napari version no.: 0.4.0


### Load image data

In [2]:
gfp = imread('/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/stacks/gfp.tif')

In [3]:
rfp = imread('/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/stacks/rfp.tif')

In [4]:
#bf = imread('/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/stacks/bf.tif')

### Load tracking data

In [3]:
with btrack.dataio.HDF5FileHandler("/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5", 'r', obj_type = "obj_type_1") as hdf:
    wt_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler("/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5", 'r', obj_type = "obj_type_2") as hdf:
    scr_tracks = hdf.tracks
print("Tracks loaded")

[INFO][2020/12/03 05:37:53 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2020/12/03 05:37:53 PM] Loading tracks/obj_type_1
[INFO][2020/12/03 05:37:55 PM] Loading objects/obj_type_1 (408973, 5) (388394 filtered: area>=100)
[INFO][2020/12/03 05:37:58 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2020/12/03 05:37:58 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2020/12/03 05:37:58 PM] Loading tracks/obj_type_2
[INFO][2020/12/03 05:37:58 PM] Loading objects/obj_type_2 (12115, 5) (8894 filtered: area>=100)
[INFO][2020/12/03 05:37:59 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5


Tracks loaded


### Isolate one track of interest

In [7]:
## isolate one target scribble track of interest
print("input scribble track of interest ID")
cell_ID = int(input())
index = [i for i, scr_tracks in enumerate(scr_tracks) if scr_tracks.ID == cell_ID][0]
target_track = scr_tracks[index]##### Show the first classification of each track
apop_index = find_apoptosis_time(target_track, index = True)

input scribble track of interest ID
17


### Filter tracks for specific events 

In [5]:
divisions = [t for t in wt_tracks if t.fate.name == "DIVIDE" ]#and t.in_frame(730)]

In [6]:
scr_apops = [t for t in scr_tracks if t.fate.name == "APOPTOSIS"]

In [ ]:
# wt_tracks = divisions
# scr_tracks = scr_apops

### Necessary coordinate shift (due to stablised images used on tracks)

In [7]:
### finding coord range of aligned images, coords switched already
align_x_range, align_y_range = gfp.shape[2], gfp.shape[1]
### finding maximum extent of tracking coords
tracks_x_range = round(max([max(track.x) for track in wt_tracks]))
tracks_y_range = round(max([max(track.y) for track in wt_tracks])) + 2 ## sort this lazy hack out later

### coord switch
tmp = tracks_y_range
tracks_y_range = tracks_x_range
tracks_x_range = tmp

print("tracks range:", (tracks_x_range), (tracks_y_range))
print("aligned image range:", (align_x_range), (align_y_range))

shift_x = int((align_x_range - tracks_x_range)/2)
shift_y = int((align_y_range - tracks_y_range)/2)

print("shift in x and y:", shift_x, shift_y)

tracks range: 1600 1200
aligned image range: 1739 1377
shift in x and y: 69 88


### Apply coordinate transpose and shift

In [8]:
wt_data, properties, graph = tracks_to_napari(wt_tracks, ndim = 2)
scr_data, properties, graph = tracks_to_napari(scr_tracks, ndim = 2)

tmp = wt_data[:,2].copy() ## copy the true_y coord
wt_data[:,2] = wt_data[:,3]  ##assign the old_y coord as the true_x
wt_data[:,3] = tmp ## assign the old_x as true_y

wt_data[:,2] += shift_y ## TRUE_Y (vertical axis)
wt_data[:,3] += shift_x ## TRUE_X (horizontal axis)

tmp = scr_data[:,2].copy()
scr_data[:,2] = scr_data[:,3]
scr_data[:,3] = tmp

scr_data[:,2] += shift_y ## TRUE_Y (vertical axis)
scr_data[:,3] += shift_x ## TRUE_X (horizontal axis)


### Define reference points if needed

In [9]:
### add_points needs to be txy
apop_event = target_track.t[apop_index], target_track.x[apop_index]+shift_y, target_track.y[apop_index]+shift_x ## with transposed shift
print("txy of apoptotic event:", apop_event)

txy of apoptotic event: (730, 638.5511474609375, 1158.223876953125)


### Start napari

In [10]:
with napari.gui_qt():
    viewer = napari.Viewer()
    
    #viewer.add_image(bf)
    viewer.add_image(gfp, name="gfp", blending = "additive", colormap = "green")
    viewer.add_image(rfp, name="rfp", blending = "additive", colormap = "red")
    
    viewer.add_tracks(wt_data)
    viewer.add_tracks(scr_data)
    #viewer.add_points(apop_event)
    